In [1]:
import logging

logging.basicConfig(level=logging.DEBUG)

In [2]:
import os

os.environ['CUDA_VISIBLE_DEVICES'] = ''

In [3]:
import sys

SOURCE_DIR = os.path.dirname(os.path.dirname(os.path.dirname(os.path.abspath(__name__))))
SOURCE_DIR
sys.path.insert(0, SOURCE_DIR)

In [4]:
%%time

import malaya
malaya

DEBUG:tensorflow:Falling back to TensorFlow client; we recommended you install the Cloud TPU client directly with pip install cloud-tpu-client.
DEBUG:h5py._conv:Creating converter from 7 to 5
DEBUG:h5py._conv:Creating converter from 5 to 7
DEBUG:h5py._conv:Creating converter from 7 to 5
DEBUG:h5py._conv:Creating converter from 5 to 7
DEBUG:git.cmd:Popen(['git', 'version'], cwd=/home/husein/dev/malaya/misc/normalize-twitter, universal_newlines=False, shell=None, istream=None)
DEBUG:git.cmd:Popen(['git', 'version'], cwd=/home/husein/dev/malaya/misc/normalize-twitter, universal_newlines=False, shell=None, istream=None)


CPU times: user 3.29 s, sys: 3.33 s, total: 6.62 s
Wall time: 2.4 s


<module 'malaya' from '/home/husein/dev/malaya/malaya/__init__.py'>

In [5]:
lm = malaya.language_model.kenlm(model = 'bahasa-wiki-news')

DEBUG:urllib3.connectionpool:Starting new HTTPS connection (1): huggingface.co:443
DEBUG:urllib3.connectionpool:https://huggingface.co:443 "HEAD /huseinzol05/language-model-bahasa-wiki-news/resolve/main/model.klm HTTP/1.1" 302 0


In [6]:
stemmer = malaya.stem.deep_model(model = 'noisy')

DEBUG:urllib3.connectionpool:Starting new HTTPS connection (1): huggingface.co:443
DEBUG:urllib3.connectionpool:https://huggingface.co:443 "HEAD /huseinzol05/stem-v2-lstm-bahdanau-noisy/resolve/main/model.pb HTTP/1.1" 302 0
DEBUG:urllib3.connectionpool:Starting new HTTPS connection (1): huggingface.co:443
DEBUG:urllib3.connectionpool:https://huggingface.co:443 "HEAD /huseinzol05/bpe/resolve/main/stemmer-v2.yttm HTTP/1.1" 200 0
DEBUG:malaya_boilerplate.frozen_graph:use_tensorrt: False
DEBUG:malaya_boilerplate.frozen_graph:tensorrt_precision_mode: FP32
DEBUG:malaya_boilerplate.frozen_graph:precision_mode: FP32
DEBUG:malaya_boilerplate.frozen_graph:t5_graph: False
DEBUG:malaya_boilerplate.frozen_graph:glowtts_graph: False
DEBUG:malaya_boilerplate.frozen_graph:glowtts_multispeaker_graph: False
2022-12-03 15:30:04.875205: I tensorflow/core/platform/cpu_feature_guard.cc:142] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instruct

In [7]:
corrector = malaya.spelling_correction.probability.load(language_model = lm, replace_augmentation = True,
                                                       stemmer = stemmer, maxlen = 12, minlen = 2)

INFO:malaya_boilerplate.huggingface:downloading frozen huseinzol05/v27-preprocessing/bm_1grams.json
DEBUG:urllib3.connectionpool:Starting new HTTPS connection (1): huggingface.co:443
DEBUG:urllib3.connectionpool:https://huggingface.co:443 "HEAD /huseinzol05/v27-preprocessing/resolve/main/bm_1grams.json HTTP/1.1" 200 0


In [8]:
normalizer = malaya.normalize.normalizer(corrector, stemmer, date = False)

In [9]:
normalizer._tokenizer(t)

NameError: name 't' is not defined

In [15]:
segmenter = malaya.segmentation.huggingface()
segmenter_func = lambda x: segmenter.generate([x], max_length = 256)[0]

DEBUG:urllib3.connectionpool:Starting new HTTPS connection (1): huggingface.co:443
DEBUG:urllib3.connectionpool:https://huggingface.co:443 "HEAD /mesolitica/finetune-segmentation-t5-tiny-standard-bahasa-cased/resolve/main/tokenizer_config.json HTTP/1.1" 200 0
DEBUG:urllib3.connectionpool:Starting new HTTPS connection (1): huggingface.co:443
DEBUG:urllib3.connectionpool:https://huggingface.co:443 "HEAD /mesolitica/finetune-segmentation-t5-tiny-standard-bahasa-cased/resolve/main/config.json HTTP/1.1" 200 0


In [16]:
text_scorer = lambda x: lm.score(x)

In [17]:
t = '14:14hrs Kerja penyelenggaraan KM 83.5- KM 84.5 arah Utara dari Ayer Hitam ke Yong Peng. Lorong kiri ditutup. Trafi https://t.co/iP2jEdY4Ib'

In [34]:
normalized = normalizer.normalize(
            t, normalize_hingga = False, normalize_cardinal = False,
            normalize_ordinal = False, normalize_pada_hari_bulan = False,
            normalize_fraction = False, normalize_money = False, normalize_date = False,
            normalize_time = False, normalize_ic = False,
            normalize_units = False,
            normalize_url = False, normalize_percent=False, normalize_telephone = False,
            text_scorer = text_scorer, segmenter = segmenter_func,
            not_a_word_threshold = 1e-9, text_scorer_window = 2, lookback=5
        )['normalize']
normalized

DEBUG:malaya.normalize:before expand_contractions: 14:14hrs Kerja penyelenggaraan KM 83.5- KM 84.5 arah Utara dari Ayer Hitam ke Yong Peng. Lorong kiri ditutup. Trafi https://t.co/iP2jEdY4Ib
DEBUG:malaya.normalize:after expand_contractions: 14:14hrs Kerja penyelenggaraan KM 83.5- KM 84.5 arah Utara dari Ayer Hitam ke Yong Peng. Lorong kiri ditutup. Trafi https://t.co/iP2jEdY4Ib
DEBUG:malaya.normalize:tokenized: ['14:14hrs', 'Kerja', 'penyelenggaraan', 'KM', '83.5', '-', 'KM', '84.5', 'arah', 'Utara', 'dari', 'Ayer', 'Hitam', 'ke', 'Yong', 'Peng', '.', 'Lorong', 'kiri', 'ditutup', '.', 'Trafi', 'https://t.co/iP2jEdY4Ib']
DEBUG:malaya.normalize:before normalize_text: 14:14hrs Kerja penyelenggaraan KM 83.5 - KM 84.5 arah Utara dari Ayer Hitam ke Yong Peng . Lorong kiri ditutup . Trafi https://t.co/iP2jEdY4Ib
DEBUG:malaya.normalize:after normalize_text: 14:14hrs Kerja penyelenggaraan KM 83.5 - KM 84.5 arah Utara dari Ayer Hitam ke Yong Peng . Lorong kiri ditutup . Trafi https://t.co/iP2jEd

DEBUG:malaya.normalize:index: 11, word: Ayer, condition not in money and date
DEBUG:malaya.normalize:index: 11, word: Ayer, condition text_scorer is not None
DEBUG:malaya.normalize:index: 11, word: Ayer, lower: utara dari ayer Hitam ke , normal: utara dari Ayer Hitam ke , lower_score: -23.667076110839844, title_score: -16.45641326904297, normal_score: -16.45641326904297
DEBUG:malaya.normalize:index: 11, word: Ayer, condition titled
DEBUG:malaya.normalize:index: 12, word: Hitam, queue: ['pukul 01.38:39', 'kerja', 'penyelenggaraan', 'KM', '83.5', '-', 'KM', '84.5', 'arah', 'utara', 'dari', 'Ayer']
DEBUG:malaya.normalize:index: 12, word: Hitam, score: 0.000754400895654938, text_scorer is not None
DEBUG:malaya.normalize:index: 12, word: Hitam, condition not in money and date
DEBUG:malaya.normalize:index: 12, word: Hitam, condition text_scorer is not None
DEBUG:malaya.normalize:index: 12, word: Hitam, lower: dari Ayer hitam ke Yong , normal: dari Ayer Hitam ke Yong , lower_score: -22.882148

DEBUG:malaya.normalize:right_yesterdaydatetodaytime_: []


'pukul 01.38:39 kerja penyelenggaraan KM 83.5 - KM 84.5 arah utara dari Ayer Hitam ke Yong Peng . lorong kiri ditutup . Trafi https://t.co/iP2jEdY4Ib'

In [31]:
segmenter_func('14hrs')

DEBUG:malaya.torch_model.huggingface:generate, strings: ['14hrs']


'14hrs'

In [22]:
from malaya.text.tatabahasa import (
    date_replace,
    consonants,
    sounds,
    bulan,
)
from malaya.text.function import (
    is_laugh,
    is_mengeluh,
    multireplace,
    case_of,
    PUNCTUATION,
)

In [24]:
word = '14:14hrs'
word = multireplace(word, date_replace)
word

'14:14hrs'

In [33]:
dateparser.parse('14:14 hours', settings={'TIMEZONE': 'GMT+8'})

datetime.datetime(2022, 12, 3, 1, 35, 37, 152859)

In [19]:
normalizer._tokenizer(t)

['14:14hrs',
 'Kerja',
 'penyelenggaraan',
 'KM',
 '83.5',
 '-',
 'KM',
 '84.5',
 'arah',
 'Utara',
 'dari',
 'Ayer',
 'Hitam',
 'ke',
 'Yong',
 'Peng',
 '.',
 'Lorong',
 'kiri',
 'ditutup',
 '.',
 'Trafi',
 'https://t.co/iP2jEdY4Ib']

In [21]:
t

'14:14hrs Kerja penyelenggaraan KM 83.5- KM 84.5 arah Utara dari Ayer Hitam ke Yong Peng. Lorong kiri ditutup. Trafi https://t.co/iP2jEdY4Ib'